# Lesson 10: Type Safety

**PART 1: The problem with nested properties**

When accessing nested optional properties, you need multiple checks.

This creates deeply nested if statements.

The code becomes hard to read and maintain.

In [9]:
interface User {
  name?: string;
  email: string;
  groups?: string[];
}

interface Session {
  user?: User;
}

const getUserNameLongWay = (session: Session): string => {
  if (session.user) {
    if (session.user.name) {
      return session.user.name;
    }
  }
  return 'Guest';
}

const emptySession: Session = {};
const userWithoutName: Session = { user: { email: 'unknown@example.com' } };
const activeSession: Session = { user: { name: 'Alice', email: 'alice@example.com' } };

`Long way: ${getUserNameLongWay(emptySession)}, ${getUserNameLongWay(userWithoutName)}, ${getUserNameLongWay(activeSession)}`

"Long way: Guest, Guest, Alice"

**PART 2: Optional chaining - the solution**

The `?.` operator safely accesses nested properties.

If any part is null or undefined, it returns undefined instead of crashing.

This makes code much cleaner and safer.

In [10]:
const getUserName = (session: Session): string => session.user?.name || 'Guest';

`Short way: ${getUserName(emptySession)}, ${getUserName(userWithoutName)}, ${getUserName(activeSession)}`

"Short way: Guest, Guest, Alice"

In the code above ...

-  session can't be undefined 
    => no need to put a ? after session.
  
-  user is optional and therefore might be undefined 
    => ? is required after session.user to (conditionally) access properties of user such as name
  
-  name is optional and therefore might be undefined
    => ? isn't required after name because there is no attempt to access properties of name


**PART 3: nested optional chaining**

Here is an example of where multiple `?.` operators exist in the same expression

In [11]:
type PossibleSession = Session | null | undefined;

const getPossibleUserName = (session: PossibleSession): string => session?.user?.name || 'Guest';

const empty = {};
const liveSession = { user: { name: 'Alice', email: 'alice@example.com' } };

`Short way: ${getPossibleUserName(empty)}, ${getPossibleUserName(liveSession)}`

"Short way: Guest, Alice"

**PART 4: The nullish coalescing operator**

The `??` operator provides defaults for null or undefined.

This is more precise than the `||` operator used above.

`||` treats empty strings (or 0) as false

`??` treats empty strings or 0 as true.

In the example below, we see how an empty string is preserved by the ?? operator.

In [12]:
const status = "";   // user intentionally cleared their status

const statusOr = status || "Status not found";   
const statusNullish = status ?? "No status";  

console.log(`With || ->${statusOr}<-`);  
console.log(`With ?? ->${statusNullish}<-`);  

With || ->Status not found<-
With ?? -><-


**PART 5: Optional chaining with arrays**

Use `?.[]` to safely access array elements.

Works even if the array doesn't exist or is empty.

Returns undefined instead of throwing an error.

In [13]:
function getFirstGroup(session: Session): string | undefined {
  return session.user?.groups?.[0];
}

const noGroups: Session = { user: { name: 'Alice', email: 'alice@example.com' } };
const hasGroups: Session = { user: { name: 'Bob', email: 'bob@example.com', groups: ['admin'] } };

`No groups: ${getFirstGroup(noGroups)}, Has groups: ${getFirstGroup(hasGroups)}`

"No groups: undefined, Has groups: admin"

**PART 6: Type narrowing**

TypeScript tracks types inside conditionals.

After checking `if (user)`, TypeScript knows user isn't null.

This is called type narrowing - the type becomes more specific.

In [14]:
const aliceEmail: User = {
      email: 'alice@example.com',
    };


processUserEmail(aliceEmail);
processUserEmail(null);

function processUserEmail(user: User | null): void {
  if (user) {
    // TypeScript knows user is not null here and allows access to its properties
    console.log(`Processing user email: ${user.email}`);
  } else {
    console.log( 'No user to process');
    // console.log(`No name: ${user.name}`);  // uncommenting this line would cause a TypeScript type error
  }
}


Processing user email: alice@example.com
No user to process


**PART 7: Non-null assertion operator**

The `!` operator tells TypeScript "this definitely exists".

Use it only when you're absolutely certain a value isn't null or undefined.

Useful right after setting a value.

In [15]:
function storeUser(session: Session, name: string): string {
  session.user = { name: name, email: `${name}@example.com` };
  return `Stored: ${session.user!.name}`;
}

const newSession: Session = {};
storeUser(newSession, 'Carol')

"Stored: Carol"

**PART 8: Type assertions with as**

The `as` keyword tells TypeScript to treat a value as a specific type.

Useful when you know more about the type than TypeScript does.

For example: when processing data returned from a JavaScript library with no types defined.

In [16]:

// Pretend this comes from some external library.
// TypeScript doesn't know what shape it returns, so it's typed as 'any'.
function imaginaryLibraryFunction() {
    return {
        name: "Alice",
        email: "alice@example.com",
    };
}

const rawData = imaginaryLibraryFunction();

const user = rawData as User; // Tell TypeScript: "I know this is a User"

`${user.name}'s email address is ${user.email} `


"Alice's email address is alice@example.com "